<a href="https://colab.research.google.com/github/kumarpriyanshu2k2/handwriting_recognition_TF_MNIST/blob/progress/MNIST_Hand_writting_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist

# Load Data, Reshape and Normalize
One important step when dealing with image data is to preprocess the data. During the preprocess step you can apply transformations to the dataset that will be fed into your convolutional neural network.

applied transformations to the data:
- Reshaped the data so that it has an extra dimension. The reason for this 
is that commonly you will use 3-dimensional arrays (without counting the batch dimension) to represent image data. The third dimension represents the color using RGB values. This data might be in black and white format so the third dimension doesn't really add any additional information for the classification process but it is a good practice regardless.


- Normalized the pixel values so that these are values between 0 and 1. this can be acheived by dividing every value in the array by the maximum.

tensors are of type `numpy.ndarray` so functions like reshape or divide are used to complete the `reshape_and_normalize` function below:

In [17]:
# reshape_and_normalize function

def reshape_and_normalize(images):
    
    # Reshaped the images to add an extra dimension
    y=images.shape+(1,)
    images=np.reshape(images,y)
    
    # Normalized pixel values
    images = images/255
    

    return images

In [18]:
(training_images, training_labels), _ = mnist.load_data() 

# Apply your function
training_images = reshape_and_normalize(training_images)

print(f"Maximum pixel value after normalization: {np.max(training_images)}\n")
print(f"Shape of training set after reshaping: {training_images.shape}\n")
print(f"Shape of one image after reshaping: {training_images[0].shape}")


Maximum pixel value after normalization: 1.0

Shape of training set after reshaping: (60000, 28, 28, 1)

Shape of one image after reshaping: (28, 28, 1)


# Callback function:
- callbacke function is used to stop training the model once 99.5% accuracy is reached to avoid overfitting the data

In [8]:
# Callback
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self,epoch,logs={}):
        if logs.get('accuracy') is not None and logs.get('accuracy') > 0.995:
            self.model.stop_training=True
            print('\n Reached 99.5% accuracy so cancelling training!') 




# The Model:
- one convolutional layer and one max pool layer is used in model

In [9]:
# convolutional model
def convolutional_model():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=(28,28,1)),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128,activation='relu'),
        tf.keras.layers.Dense(10,activation='softmax'),

        ])

    # Compile the model
    model.compile(optimizer='adam', 
                  loss='sparse_categorical_crossentropy', 
                  metrics=['accuracy']) 
        
    return model

# Training the Model:

In [10]:
# Saved untrained model
model = convolutional_model()

# Instantiate the callback class
callbacks = myCallback()

# Train model 
history = model.fit(training_images, training_labels, epochs=10, callbacks=[callbacks])

Epoch 1/10
1875/1875 [==============================] - 16s 3ms/step - loss: 0.1570 - accuracy: 0.9534
Epoch 2/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0528 - accuracy: 0.9837
Epoch 3/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0354 - accuracy: 0.9888
Epoch 4/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0229 - accuracy: 0.9928
Epoch 5/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0153 - accuracy: 0.9947
Epoch 6/10
1867/1875 [============================>.] - ETA: 0s - loss: 0.0112 - accuracy: 0.9963
 Reached 99.5% accuracy so cancelling training!
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0112 - accuracy: 0.9963


In [36]:
model.summary()
print("\n\n"+f"model was trained for {len(history.epoch)} epochs")

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 13, 13, 32)       0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 5408)              0         
                                                                 
 dense_2 (Dense)             (None, 128)               692352    
                                                                 
 dense_3 (Dense)             (None, 10)                1290      
                                                                 
Total params: 693,962
Trainable params: 693,962
Non-trainable params: 0
________________________________________________

# Test Model:

In [33]:
from google.colab import files
from keras.preprocessing import image
from PIL import Image
import cv2

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn

  img = image.load_img(path, target_size=(28, 28))
  gray = cv2.cvtColor(np.float32(img), cv2.COLOR_BGR2GRAY)
  gray = np.expand_dims(gray, 2)
  gray = np.array(gray)
  #reshaping to support our model input and normalizing
  gray = gray.reshape(1,28,28,1)
  gray = gray/255.0
  classes = model.predict([gray], batch_size=10)[0]
  print(np.argmax(classes),max(classes))


Saving Screenshot 2022-05-13 at 5.22.09 PM.jpg to Screenshot 2022-05-13 at 5.22.09 PM.jpg
5 0.9985215
